In [ ]:
from dlgo.data.parallel_processor import GoDataProcessor
from dlgo.encoders.simple import SimpleEncoder
from layers import layers

import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import os

Data Processing

In [ ]:
board_size = 19
num_classes = board_size * board_size
num_games = 100

encoder = SimpleEncoder((board_size, board_size))

processor = GoDataProcessor(encoder=encoder.name(), data_directory='data')

generator = processor.load_go_data('train', num_games, use_generator=True)
test_generator = processor.load_go_data('test', num_games, use_generator=True)

In [ ]:
class GoDatasetWrapper(Dataset):
    def __init__(self, generator, batch_size, num_classes):
        self.generator = generator.generate(batch_size, num_classes)
        self.num_samples = generator.get_num_samples()

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        # one batch
        X, y = next(self.generator)
        X = torch.tensor(X, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.long)  # class indices for CrossEntropyLoss
        return X, y


Defining the Model

In [ ]:
class BetterGoCNN(nn.Module):
    def __init__(self, board_size=19):
        super(BetterGoCNN, self).__init__()
        input_shape = (11, board_size, board_size)

        self.model = layers(input_shape)

    def forward(self, x):
        return self.model(x)
        

Training Loop

In [ ]:
model = BetterGoCNN(board_size)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

train_dataset = GoDatasetWrapper(generator, batch_size=128, num_classes=num_classes)
test_dataset = GoDatasetWrapper(test_generator, batch_size=128, num_classes=num_classes)

train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
epochs = 5
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, targets in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        inputs, targets = inputs.squeeze(0).to(device), targets.squeeze(0).to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == targets).sum().item()
        total += targets.size(0)

    accuracy = correct / total
    print(f"Train Loss: {running_loss:.4f}, Accuracy: {accuracy:.4f}")

    torch.save(model.state_dict(), f"small_model_epoch_{epoch+1}.pth")

Evaluation Loop

In [ ]:
model.eval()
correct = 0
total = 0
eval_loss = 0.0

with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.squeeze(0).to(device), targets.squeeze(0).to(device)
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        eval_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == targets).sum().item()
        total += targets.size(0)

print(f"Test Loss: {eval_loss:.4f}, Accuracy: {correct/total:.4f}")
